In [157]:
import pandas as pd
from business_models.databases.hahn import HahnDataLoader
from functools import reduce

In [ ]:
from buisn

In [2]:
with open('/home/victorshch/data/my-yql-token.txt', 'r') as f:
    yql_token = f.read().strip()

In [3]:
hahn = HahnDataLoader(yql_token)

In [9]:
hahn.yt.select_rows?

Signature:
hahn.yt.select_rows(
    query,
    timestamp=None,
    input_row_limit=None,
    output_row_limit=None,
    range_expansion_limit=None,
    fail_on_incomplete_result=None,
    verbose_logging=None,
    enable_code_cache=None,
    max_subqueries=None,
    workload_descriptor=None,
    allow_full_scan=None,
    allow_join_without_index=None,
    format=None,
    raw=None,
    execution_pool=None,
    response_parameters=None,
    client=None,
)
Docstring:
Executes a SQL-like query on dynamic table.

.. seealso:: `supported features <https://yt.yandex-team.ru/docs/description/dynamic_tables/dyn_query_language>`_

:param str query: for example "<columns> [as <alias>], ... from \[<table>\]                   [where <predicate> [group by <columns> [as <alias>], ...]]".
:param int timestamp: timestamp.
:param format: output format.
:type format: str or descendant of :class:`Format <yt.wrapper.format.Format>`
:param bool raw: don't parse response to rows.
File:      /usr/local/lib/p

In [17]:
result = hahn('select count(*) from range(`//home/taxi/home/kokokosttn/taxi-devops-data/solomon/data`)')

In [19]:
type(result)

pandas.core.frame.DataFrame

In [26]:
def make_label_filter(**label_dict):
    term_list = []
    for key, value in label_dict.items():
        term_list.append(
            f'labels["{key}"] = "{value}"'
        )
    return " and ".join(term_list)

In [27]:
make_label_filter(a=1, b=2)

'labels["a"] = "1" and labels["b"] = "2"'

In [28]:
sample_query = query_tpl.format(label_filter = make_label_filter(**{'a': '1', 'b': '2'}))

In [29]:
sample_query

'select labels, timestamps, values from range(`//home/taxi/home/kokokosttn/taxi-devops-data/solomon/data`) where labels["a"] = "1" and labels["b"] = "2"'

In [41]:
DATA_DIR = '//home/taxi/home/victorshch/solomon_data_sample_dir'

In [127]:
QUERY_TPL = 'select labels, ListFilter(ListZip(timestamps, `values`), ($x) -> {{ RETURN {ts_from} < $x.0 AND $x.0 <= {ts_to}; }}) as tuples from range(`{data_dir}`) where {label_filter}'
def make_query(data_dir, time_from, time_to, **label_dict):
    ts_from = int(time_from.timestamp())
    ts_to = int(time_to.timestamp())
    query = QUERY_TPL.format(data_dir=data_dir, ts_from=ts_from, ts_to=ts_to, label_filter=make_label_filter(**label_dict))
    return query

In [128]:
print(make_query(DATA_DIR, pd.to_datetime(0), pd.to_datetime('2021-12-31'), a=1, b=2))

select labels, ListFilter(ListZip(timestamps, `values`), ($x) -> { RETURN 0 < $x.0 AND $x.0 < 1640908800; }) as tuples from range(`//home/taxi/home/victorshch/solomon_data_sample_dir`) where labels["a"] = "1" and labels["b"] = "2"


In [69]:
pre_result = hahn(make_query(cluster='production', object='corp-clients_taxi_yandex_net_v1_cards_list_GET'))

In [142]:
pre_result

,labels,tuples
0,{'object': 'corp-clients_taxi_yandex_net_v1_ca...,"[(1635792600, 0.0)]"
1,{'object': 'corp-clients_taxi_yandex_net_v1_ca...,"[(1635792600, 0.0)]"


In [143]:
pre_result.loc[0, :]

labels    {'object': 'corp-clients_taxi_yandex_net_v1_ca...
tuples                                  [(1635792600, 0.0)]
Name: 0, dtype: object

In [138]:
def make_ts_df_from_series(s):
    multiindex_tuples = list(s['labels'].items())
    multiindex_unzipped = list(zip(*multiindex_tuples))
    multiindex = pd.MultiIndex.from_tuples([multiindex_unzipped[1]], names=multiindex_unzipped[0])
    timestamps, values = zip(*s['tuples'])
    result = pd.DataFrame(values, index=pd.to_datetime(timestamps, unit='s'), columns=multiindex)
    return result

In [144]:
df0 = make_ts_df_from_series(pre_result.loc[0, :])
df0

object,corp-clients_taxi_yandex_net_v1_cards_list_GET
sensor,ok_rps
group,dorblu_rtc_taxi_corp-clients_stable
host,cluster
project,taxi
service,dorblu
cluster,production
2021-11-01 18:50:00,0.0


In [145]:
df1 = make_ts_df_from_series(pre_result.loc[1, :])
df1

object,corp-clients_taxi_yandex_net_v1_cards_list_GET
sensor,ok_rps
group,dorblu_rtc_taxi_corp-clients_pre_stable
host,cluster
project,taxi
service,dorblu
cluster,production
2021-11-01 18:50:00,0.0


In [152]:
df0.join(df1, how='outer')

object              corp-clients_taxi_yandex_net_v1_cards_list_GET  \
sensor                                                      ok_rps   
group                          dorblu_rtc_taxi_corp-clients_stable   
host                                                       cluster   
project                                                       taxi   
service                                                     dorblu   
cluster                                                 production   
2021-11-01 18:50:00                                            0.0   

object                                                       
sensor                                                       
group               dorblu_rtc_taxi_corp-clients_pre_stable  
host                                                cluster  
project                                                taxi  
service                                              dorblu  
cluster                                          production  
2021-11-01 18:50:00                                     0.0

In [160]:
reduce(lambda left, right: left.join(right, how='outer'), [df0, df1])

object              corp-clients_taxi_yandex_net_v1_cards_list_GET  \
sensor                                                      ok_rps   
group                          dorblu_rtc_taxi_corp-clients_stable   
host                                                       cluster   
project                                                       taxi   
service                                                     dorblu   
cluster                                                 production   
2021-11-01 18:50:00                                            0.0   

object                                                       
sensor                                                       
group               dorblu_rtc_taxi_corp-clients_pre_stable  
host                                                cluster  
project                                                taxi  
service                                              dorblu  
cluster                                          production  
2021-11-01 18:50:00                                     0.0

In [161]:
def make_ts_df(yt_df):
    series_list = [make_ts_df_from_series(row[1]) for row in yt_df.iterrows()]
    return reduce(lambda left, right: left.join(right, how='outer'), series_list)

In [162]:
ts_df = make_ts_df(pre_result)

In [163]:
ts_df

object              corp-clients_taxi_yandex_net_v1_cards_list_GET  \
sensor                                                      ok_rps   
group                          dorblu_rtc_taxi_corp-clients_stable   
host                                                       cluster   
project                                                       taxi   
service                                                     dorblu   
cluster                                                 production   
2021-11-01 18:50:00                                            0.0   

object                                                       
sensor                                                       
group               dorblu_rtc_taxi_corp-clients_pre_stable  
host                                                cluster  
project                                                taxi  
service                                              dorblu  
cluster                                          production  
2021-11-01 18:50:00                                     0.0

In [60]:
multiindex_tuples = list(pre_result['labels'][0].items())
multiindex_tuples

[('cluster', 'production'),
 ('service', 'dorblu'),
 ('project', 'taxi'),
 ('host', 'cluster'),
 ('group', 'dorblu_rtc_taxi_corp-clients_stable'),
 ('sensor', 'ok_rps'),
 ('object', 'corp-clients_taxi_yandex_net_v1_cards_list_GET')]

In [64]:
multiindex_unzipped = list(zip(*multiindex_tuples))

In [66]:
multiindex = pd.MultiIndex.from_tuples([multiindex_unzipped[1]], names=multiindex_unzipped[0])
multiindex

MultiIndex([('production', 'dorblu', 'taxi', 'cluster', ...)],
           names=['cluster', 'service', 'project', 'host', 'group', 'sensor', 'object'])

In [67]:
result = pd.DataFrame(pre_result['values'][0], index=pd.to_datetime(pre_result['timestamps'][0], unit='s'), columns=multiindex)

In [68]:
result

cluster,production
service,dorblu
project,taxi
host,cluster
group,dorblu_rtc_taxi_corp-clients_stable
sensor,ok_rps
object,corp-clients_taxi_yandex_net_v1_cards_list_GET
2021-11-01 18:40:00,0.0
2021-11-01 18:45:00,0.0
2021-11-01 18:50:00,0.0
2021-11-01 18:55:00,0.0


In [164]:
def load_ts_df_from_yt(hahn, time_from, time_to, data_dir=DATA_DIR, **labels):
    pre_result = hahn(make_query(data_dir, time_from, time_to, **labels))
    return make_ts_df(pre_result)

In [165]:
%%time
ts_df = load_ts_df_from_yt(hahn, pd.to_datetime('2021-11-01 18:45:00'), pd.to_datetime('2021-11-01 18:55:00'),
                           DATA_DIR,
                           cluster='production', object='corp-clients_taxi_yandex_net_v1_cards_list_GET')

CPU times: user 108 ms, sys: 0 ns, total: 108 ms
Wall time: 15.9 s


In [166]:
ts_df

object              corp-clients_taxi_yandex_net_v1_cards_list_GET  \
sensor                                                      ok_rps   
group                          dorblu_rtc_taxi_corp-clients_stable   
host                                                       cluster   
project                                                       taxi   
service                                                     dorblu   
cluster                                                 production   
2021-11-01 18:50:00                                            0.0   

object                                                       
sensor                                                       
group               dorblu_rtc_taxi_corp-clients_pre_stable  
host                                                cluster  
project                                                taxi  
service                                              dorblu  
cluster                                          production  
2021-11-01 18:50:00                                     0.0

In [129]:
pre_result = hahn(make_query(DATA_DIR, pd.to_datetime('2021-11-01 18:45:00'), pd.to_datetime('2021-11-01 18:55:00'), cluster='production', object='corp-clients_taxi_yandex_net_v1_cards_list_GET'))

In [130]:
pre_result

,labels,tuples
0,{'object': 'corp-clients_taxi_yandex_net_v1_ca...,"[(1635792600, 0.0)]"
1,{'object': 'corp-clients_taxi_yandex_net_v1_ca...,"[(1635792600, 0.0)]"
